# Q1

参考数据中给出了全国区划.gpkg（包含“市”图层）和zhejiang_pop.csv两个文件,

请根据这两个文件，输出一个描述浙江省各市人口数据的geojson矢量文件。

> _HINT_ :  GeoPandas继承了Pandas的Contact, Join, Merge等方法

In [ ]:
import geopandas as gpd 
import pandas as pd

In [ ]:
# 读取全国区划，获取浙江部分
data_convince = gpd.read_file('data/全国区划.gpkg', layer='市', encoding="utf-8")
data_zhejiang = data_convince.loc[(data_convince['省'] == '浙江省')]

# 读取人口，并合并
data_population = gpd.read_file('data/zhejiang_pop.csv', encoding="gb2312")
data_zhejiang_population = pd.merge(data_zhejiang, data_population, left_on='市', right_on='地级市')
data_zhejiang_population.rename(columns={"geometry_x": "geometry"}, inplace=True)
zhejiand_pop = data_zhejiang_population.loc[:, [True, True, True, True, True, True, False, True, False]]

In [ ]:
# 处理结果
zhejiand_pop

In [ ]:
zhejiand_pop.to_file('data/zhejiang_pop.geojson', driver='GeoJSON', encoding='utf-8')

# Q2

基于Q1的结果，选择合适的投影坐标系，制作“浙江省各市人口密度图”，要求包含一定的制图要素

In [ ]:
import pyproj
import matplotlib.pyplot as plt

In [ ]:
zhejiand_pop.to_crs(pyproj.CRS.from_user_input('EPSG:4236'))
zhejiand_pop['人口数'] = zhejiand_pop['人口数'].map(lambda x: int(x))
zhejiand_pop['area'] = zhejiand_pop.to_crs(epsg=4326).area * 10000
zhejiand_pop['人口密度'] = zhejiand_pop['人口数'] / zhejiand_pop['area']

In [ ]:
%matplotlib widget
plt.rcParams['font.family'] = 'SimHei'
fig = plt.figure(figsize=(7, 8))
ax = fig.add_axes((0.1, 0.1, 0.8, 0.8))
ax.set_title('浙江省各市人口分布图')
zhejiand_pop.plot(ax = ax, edgecolor = 'black',
                  linewidth = 0.5,
                  column = '人口密度',
                  legend = True,
                  legend_kwds = {
                      'title': '人口密度',
                      'loc': 'lower right',
                      'ncol': 1,
                  },
                  cmap = 'GnBu',
                  scheme = 'NaturalBreaks',
                  k = 5,
                 )
zhejiand_pop.apply(lambda x: ax.annotate(text=x['市'],
                                          xy=x.geometry.centroid.coords[0], 
                                          ha='center',
                                          color = '#ef883b'
                                        ), axis=1)